# Iterating simulation function

In [1]:
#imports
import numpy as np
import pandas as pd
import toytree

In [37]:
#The simulation function
def simcom(sr, pa = 0, df = True, verbose = False):
    """
    Simulate a community from the species pool under different phylogenetic structure assumptions

    Parameters:
    ---
    sr: int; species richness
        - Must be int < len(species)
    pa: float (-1 to 1); phylogenetic assumption (default = 0)
        - -1 = related species are least likely to co-occur
        - 0 = no phylogenetic structure
        - 1 = related species are most likely to co-occur

    Return:
    ---
    matrix of species presence/absence
    
    """

    #Get species distance matrix
    dm = sp_tree.distance.get_tip_distance_matrix(df = True)

    #Start community with a random species
    comm = []
    start = np.random.choice(dm.index)
    comm.append(start)
    if verbose:
        print(f"The starting species is {comm}")

    #Making a list of unselected species
    unused = set(dm.index)
    unused.remove(start)

    #Creating probability weights (min and max are 0.5 for pa = 0)
    max_p = abs(pa)/2 + 0.5
    min_p = 1 - max_p
    if verbose:
        print(f"min = {min_p}, max = {max_p}")
    
    #Select species from the distance matrix
    for sp in range((sr - 1)):
        #stop if no more species remain
        if not unused:
            raise Exception("Species richness must be less than or equal to the species pool.")
            break

        #get distances for current species
        current_sp = comm[-1]
        sp_dists = dm.loc[current_sp].copy()

        #setting already selected species distances to zero
        for name in dm.index:
            if name not in unused:
                sp_dists[name] = 0
                
        #finding min and max distances
        d_min, d_max = sp_dists.min(), sp_dists.max()
        if verbose:
            print(f"The min distance = {d_min}, and the max distance = {d_max}")

        #apply weighted probabilities to distances
        if sp_dists.sum() > 0: #ensure there are species left to collect
            if pa > 0: #smaller distances more likely
                probabilities = sp_dists.map(lambda x: 0 if x == 0 else max_p - (x - d_min) * (max_p - min_p) / (d_max - d_min))
            elif pa == 0: #no phylogenetic structure
                probabilities = sp_dists.map(lambda x: 0 if x == 0 else 0.5)
            elif pa < 0: #greater distances more likely
                probabilities = sp_dists.map(lambda x: 0 if x == 0 else min_p + (x - d_min) * (max_p - min_p) / (d_max - d_min))
            else:
                raise Exception("Uh oh, something went wrong with the weighted probabilities")

            #choose one of the species distance groups
            prob_uniq = probabilities.unique() #unique probabilities
            if pa != 0:
                prob_scale = []
                for p in prob_uniq:
                    new_prob = p / sum(prob_uniq) #scale probabilities so they sum to 1
                    prob_scale.append(new_prob)
                next_sp_prob = np.random.choice(prob_uniq, p = prob_scale) #pick a probability group by scaled probability
            else:
                next_sp_prob = 0.5
            if verbose:
                print(f"The next species probability is {next_sp_prob}")
                print(probabilities)
            
            #selecting the next species from the corresponding probability group
            possible_sp = probabilities[probabilities == next_sp_prob]
            next_sp = np.random.choice(possible_sp.index)
            if verbose:
                print(f"The next species is {next_sp}")

            #add next species and remove it from list of possible species
            comm.append(next_sp)
            unused.remove(next_sp)
        
        else:
            print("No more species!")
            break
    if verbose:
        print(comm)

    #Creating the presence/absence matrix
    comm_pa_list = []
    for sp in species:
        if sp in comm:
            comm_pa_list.append(1)
        else:
            comm_pa_list.append(0)
    if verbose:
        print(comm_pa_list)
    comm_pa = pd.DataFrame([comm_pa_list], columns = species)

    if not df:
        comm_pa = comm_pa.to_numpy()

    return comm_pa
 

In [3]:
#Creating the species pool (100 species)
sp_tree = toytree.rtree.baltree(100, random_names=False) #create the tree
species = sp_tree.get_tip_labels() #list of species in the pool

In [39]:
#Testing the simulation function works
simcom(sr = 20, pa = -0.7)

<bound method DataFrame.to_numpy of    r0  r1  r2  r3  r4  r5  r6  r7  r8  r9  ...  r90  r91  r92  r93  r94  r95  \
0   1   1   1   0   0   1   0   0   0   0  ...    0    0    0    0    0    0   

   r96  r97  r98  r99  
0    0    0    0    0  

[1 rows x 100 columns]>

## Drafting the iterative function

In [34]:
def sitesp(sr, pa = 0, nsites = 10, df = True):
    """
    Simulate a site by species matrix under a specified phylogenetic assumption

    Parameters:
    ---
    nsites: int; number of rows (sites)
    sr: int; species richness
        - Must be int < len(species)
    pa: float (-1 to 1); phylogenetic assumption (default = 0)
        - -1 = related species are least likely to co-occur
        - 0 = no phylogenetic structure
        - 1 = related species are most likely to co-occur

    Return:
    ---
    matrix of species presence/absence; rows = sites, columns = species (pandas df default)
    
    """
    ssm = pd.DataFrame([], columns = species) #make an empty dataframe with species as columns
    for s in range(nsites):
        site = simcom(sr, pa)
        ssm = pd.concat([ssm, site], ignore_index = True) #add new community row to dataframe

    if not df:
        ssm = ssm.to_numpy() #changes pandas data frame output to numpy array
    return ssm

In [36]:
#Testing the function
sitesp(25, nsites = 50, df = True)

,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9,...,r90,r91,r92,r93,r94,r95,r96,r97,r98,r99
0,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,1,1,0,0,1,0,0,1,1,0,...,0,0,1,0,0,0,0,0,1,0
3,0,0,1,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,1,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
6,0,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
7,1,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,1,1,1,1,0,0,0,0
9,1,0,0,1,1,0,0,1,1,0,...,0,0,1,0,0,1,0,0,0,0


Messy work:

In [22]:
test = pd.DataFrame([], columns = species)
test_com = simcom(50, 0)
pd.concat([test, test_com])

,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9,...,r90,r91,r92,r93,r94,r95,r96,r97,r98,r99
0,1,1,0,1,1,0,1,1,0,1,...,1,1,1,0,1,0,1,0,1,1
